In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
import keras.backend as Kr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from numpy import exp
import matplotlib.pyplot as plt
import matplotlib;matplotlib.rcParams['figure.figsize'] = (10,7)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm.keras import TqdmCallback

num_sim = 100

def model_function(df_train, phi, y_train, sim_iteration):

    model = Sequential()
    model.add(Dense(100, input_dim = phi.shape[1],kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))

    optimizer = keras.optimizers.Adam(learning_rate=0.0012)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    
    callbacks = [EarlyStopping(monitor='val_loss', patience=200),
             ModelCheckpoint(filepath='indicator_kriging.h5',
                             monitor='val_loss', save_best_only=True),
             TqdmCallback(verbose=1)]

    print("##### End of warning messages ######")
    print('<<<<<<<<<<<<<<<< Fitting DNN-model for %4d-th simulation >>>>>>>>>>>>>>>>>'%(sim_iteration + 1))
    result = model.fit(phi, y_train, callbacks=callbacks,
               validation_split = 0.1, epochs = 500, batch_size = 64, verbose = 0)

    model = keras.models.load_model('indicator_kriging.h5')
    return model

In [2]:
def predict_with_uncertainty(models, X):
    preds = np.array([m.predict(X).flatten() for m in models])
    mean_pred = np.mean(preds, axis=0)
    var_pred = np.var(preds, axis=0)
    return mean_pred, var_pred

from sklearn.neighbors import NearestNeighbors

def get_aleatoric_variance(s_train, s_test, residuals, k=40):
    nbrs = NearestNeighbors(n_neighbors=k).fit(s_train)
    _, indices = nbrs.kneighbors(s_test)
    return np.array([np.mean(residuals[idx]) for idx in indices])

def Deepkriging(g_val, h_val):
    mae_list, mse_list, picp_list, al_list, time_list = [], [], [], [], []
    n_members = 20

    for sim in range(num_sim):

        print(f"\n--- Simulation {sim+1}/{num_sim} ---")

        train_file = f"synthetic_data_simulations/training_data/Tgh_Gaussian_3600_classification_g{g_val}_h{h_val}_{sim+1}train.csv"
        test_file  = f"synthetic_data_simulations/testing_data/Tgh_Gaussian_3600_classification_g{g_val}_h{h_val}_{sim+1}test.csv"
        df_train = pd.read_csv(train_file)
        df_test = pd.read_csv(test_file)

        s_train = np.vstack((df_train["x"], df_train["y"])).T
        s_test = np.vstack((df_test["x"], df_test["y"])).T

        # Basis + covariates
        num_basis = [5**2, 7**2, 11**2]
        knots_1d = [np.linspace(0, 1, int(np.sqrt(i))) for i in num_basis]

        def compute_basis(s, num_basis, knots_1d):
            N = len(s)
            phi = np.zeros((N, sum(num_basis)))
            K = 0
            for res in range(len(num_basis)):
                theta = 1 / np.sqrt(num_basis[res]) * 2.5
                k1, k2 = np.meshgrid(knots_1d[res], knots_1d[res])
                knots = np.column_stack((k1.flatten(), k2.flatten()))
                for i in range(num_basis[res]):
                    d = np.linalg.norm(s - knots[i, :], axis=1) / theta
                    phi[:, i + K] = np.where((d >= 0) & (d <= 1),
                                             (1 - d) ** 6 * (35 * d**2 + 18 * d + 3) / 3, 0)
                K += num_basis[res]
            return phi

        phi_train = compute_basis(s_train, num_basis, knots_1d)
        phi_test = compute_basis(s_test, num_basis, knots_1d)

        y_train = df_train["tgh"].values
        y_test = df_test["tgh"].values

        # Time the ensemble training
        start_time = time.time()
        #ensemble = [model_function(df_train, phi_train, y_train, sim * 100 + i) for i in range(n_members)]
        ensemble = []
        for i in range(n_members):
            phi_sub, _, y_sub, _ = train_test_split(
                phi_train, y_train, test_size=0.2, random_state=sim * 100 + i)

            try:
                model = model_function(df_train, phi_sub, y_sub, sim * 100 + i)
                ensemble.append(model)
            except Exception as e:
                print(f"[Warning] Model {i} in Sim {sim+1} failed: {str(e)}")
                continue


        train_time = time.time() - start_time
        time_list.append(train_time)

        # Predict on test
        mean_pred, var_pred = predict_with_uncertainty(ensemble, phi_test)

        # Predict on train (for aleatoric)
        train_preds = np.array([m.predict(phi_train).flatten() for m in ensemble])
        train_mean_pred = np.mean(train_preds, axis=0)
        train_var_pred = np.var(train_preds, axis=0)
        residuals = (y_train - train_mean_pred) ** 2 - train_var_pred
        residuals[residuals < 0] = 0  # avoid negative variance

        # Aleatoric via neighbor residuals
        r_pred = get_aleatoric_variance(s_train, s_test, residuals, k=40)

        # Final PI: mean ± 1.96 * sqrt(epistemic + aleatoric)
        std_total = np.sqrt(var_pred + r_pred)
        lower = mean_pred - 1.96 * std_total
        upper = mean_pred + 1.96 * std_total

        # Metrics
        mae = np.mean(np.abs(y_test - mean_pred))
        mse = np.mean((y_test - mean_pred) ** 2)
        picp = np.mean((y_test >= lower) & (y_test <= upper))
        al = np.mean(upper - lower)

        print(f"[Sim {sim+1}] g={g_val}, h={h_val} | MAE={mae:.3f}, MSE={mse:.3f}, PICP={picp:.3f}, AL={al:.3f}, Time={train_time:.2f}s")
        mae_list.append(mae)
        mse_list.append(mse)
        picp_list.append(picp)
        al_list.append(al)

    # Save summary
    df = pd.DataFrame({
        "MAE": mae_list,
        "MSE": mse_list,
        "PICP": picp_list,
        "AL": al_list,
        "Train_Time": time_list
    })
    df.to_csv("results_with_interval.csv", index=False)

    print(f"\nAverage Training Time over {num_sim} simulations: {np.nanmean(time_list):.2f} seconds")

In [3]:
def Deepkriging(g_val, h_val):
    mae_list, mse_list, picp_list, al_list, time_list = [], [], [], [], []
    n_members = 20

    for sim in range(num_sim):
        print(f"\n--- Simulation {sim+1}/{num_sim} ---")

        train_file = f"synthetic_data_simulations/training_data/Tgh_Gaussian_3600_classification_g{g_val}_h{h_val}_{sim+1}train.csv"
        test_file  = f"synthetic_data_simulations/testing_data/Tgh_Gaussian_3600_classification_g{g_val}_h{h_val}_{sim+1}test.csv"
        df_train = pd.read_csv(train_file)
        df_test = pd.read_csv(test_file)

        s_train = np.vstack((df_train["x"], df_train["y"])).T
        s_test = np.vstack((df_test["x"], df_test["y"])).T

        # Basis + covariates
        num_basis = [5**2, 7**2, 11**2]
        knots_1d = [np.linspace(0, 1, int(np.sqrt(i))) for i in num_basis]

        def compute_basis(s, num_basis, knots_1d):
            N = len(s)
            phi = np.zeros((N, sum(num_basis)))
            K = 0
            for res in range(len(num_basis)):
                theta = 1 / np.sqrt(num_basis[res]) * 2.5
                k1, k2 = np.meshgrid(knots_1d[res], knots_1d[res])
                knots = np.column_stack((k1.flatten(), k2.flatten()))
                for i in range(num_basis[res]):
                    d = np.linalg.norm(s - knots[i, :], axis=1) / theta
                    phi[:, i + K] = np.where((d >= 0) & (d <= 1),
                                             (1 - d) ** 6 * (35 * d**2 + 18 * d + 3) / 3, 0)
                K += num_basis[res]
            return phi

        phi_all = compute_basis(s_train, num_basis, knots_1d)
        phi_test = compute_basis(s_test, num_basis, knots_1d)
        y_all = df_train["tgh"].values
        y_test = df_test["tgh"].values

        # === Split training data into model training and residual estimation ===
        phi_model, phi_resid, y_model, y_resid, s_model, s_resid = train_test_split(
            phi_all, y_all, s_train, test_size=0.5, random_state=sim
        )

        # === Train ensemble ===
        start_time = time.time()
        ensemble = []
        for i in range(n_members):
            phi_sub, _, y_sub, _ = train_test_split(
                phi_model, y_model, test_size=0.1, random_state=sim * 100 + i)

            try:
                model = model_function(df_train, phi_sub, y_sub, sim * 100 + i)
                ensemble.append(model)
            except Exception as e:
                print(f"[Warning] Model {i} in Sim {sim+1} failed: {str(e)}")
                continue
        train_time = time.time() - start_time
        time_list.append(train_time)

        # === Predict on test set ===
        mean_pred, var_pred = predict_with_uncertainty(ensemble, phi_test)

        # === Predict on residual estimation set ===
        train_preds = np.array([m.predict(phi_resid).flatten() for m in ensemble])
        train_mean_pred = np.mean(train_preds, axis=0)
        train_var_pred = np.var(train_preds, axis=0)
        residuals = (y_resid - train_mean_pred) ** 2 - train_var_pred
        residuals[residuals < 0] = 0  # avoid negative variance

        # === Estimate aleatoric variance from neighbors of s_resid ===
        r_pred = get_aleatoric_variance(s_resid, s_test, residuals, k=40)

        # === Final PI ===
        std_total = np.sqrt(var_pred + r_pred)
        lower = mean_pred - 1.96 * std_total
        upper = mean_pred + 1.96 * std_total

        # === Metrics ===
        mae = np.mean(np.abs(y_test - mean_pred))
        mse = np.mean((y_test - mean_pred) ** 2)
        picp = np.mean((y_test >= lower) & (y_test <= upper))
        al = np.mean(upper - lower)

        print(f"[Sim {sim+1}] g={g_val}, h={h_val} | MAE={mae:.3f}, MSE={mse:.3f}, PICP={picp:.3f}, AL={al:.3f}, Time={train_time:.2f}s")
        mae_list.append(mae)
        mse_list.append(mse)
        picp_list.append(picp)
        al_list.append(al)

    # === Save summary ===
    df = pd.DataFrame({
        "MAE": mae_list,
        "MSE": mse_list,
        "PICP": picp_list,
        "AL": al_list,
        "Train_Time": time_list
    })
    df.to_csv("results_with_interval.csv", index=False)

    print(f"\nAverage Training Time over {num_sim} simulations: {np.nanmean(time_list):.2f} seconds")


In [5]:
Deepkriging(0,0)


--- Simulation 1/100 ---


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

##### End of warning messages ######
<<<<<<<<<<<<<<<< Fitting DNN-model for    1-th simulation >>>>>>>>>>>>>>>>>


0epoch [00:00, ?epoch/s]

...

0batch [00:00, ?batch/s]

##### End of warning messages ######
<<<<<<<<<<<<<<<< Fitting DNN-model for 9919-th simulation >>>>>>>>>>>>>>>>>


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

##### End of warning messages ######
<<<<<<<<<<<<<<<< Fitting DNN-model for 9920-th simulation >>>>>>>>>>>>>>>>>
51/51 [==============================] - 0s 427us/step
[Sim 100] g=0, h=0 | MAE=0.142, MSE=0.032, PICP=0.964, AL=0.767, Time=200.90s

Average Training Time over 100 simulations: 209.03 seconds


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("results_with_interval.csv")
n = len(df)

train_time_mean = np.mean(df['Train_Time'])
mae_mean = np.mean(df['MAE'])
mae_se = np.std(df['MAE'], ddof=1) / np.sqrt(n)

picp_mean = np.mean(df['PICP'])
picp_se = np.std(df['PICP'], ddof=1) / np.sqrt(n)

al_mean = np.mean(df['AL'])
al_se = np.std(df['AL'], ddof=1) / np.sqrt(n)

print("Train_Time mean:", "{:.3f}".format(train_time_mean))
print("MAE mean:", "{:.3f}".format(mae_mean))
print("MAE SE:", "{:.3f}".format(mae_se))
print("PICP mean:", "{:.3f}".format(picp_mean))
print("PICP SE:", "{:.3f}".format(picp_se))
print("AL mean:", "{:.3f}".format(al_mean))
print("AL SE:", "{:.3f}".format(al_se))


Train_Time mean: 209.034
MAE mean: 0.150
MAE SE: 0.001
PICP mean: 0.949
PICP SE: 0.001
AL mean: 0.763
AL SE: 0.003


In [11]:
np.mean(df['PICP'])

0.8095

In [5]:
def Deepkriging_mae(g_val, h_val):
    mae_list = []
    mse_list = []
    time_list = []
    
    for sim in range(num_sim):
        train_file = f"synthetic_data_simulations/training_data/Tgh_Gaussian_3600_classification_g{g_val}_h{h_val}_{sim+1}train.csv"
        test_file  = f"synthetic_data_simulations/testing_data/Tgh_Gaussian_3600_classification_g{g_val}_h{h_val}_{sim+1}test.csv"
        df_train = pd.read_csv(train_file)
        df_test = pd.read_csv(test_file)
        
        s_train = np.vstack((df_train["x"], df_train["y"])).T
        s_test = np.vstack((df_test["x"], df_test["y"])).T

        num_basis = [5**2, 7**2, 11**2]
        knots_1d = [np.linspace(0, 1, int(np.sqrt(i))) for i in num_basis]

        def compute_basis(s, num_basis, knots_1d):
            N = len(s)
            phi = np.zeros((N, sum(num_basis)))
            K = 0
            for res in range(len(num_basis)):
                theta = 1 / np.sqrt(num_basis[res]) * 2.5
                k1, k2 = np.meshgrid(knots_1d[res], knots_1d[res])
                knots = np.column_stack((k1.flatten(), k2.flatten()))
                for i in range(num_basis[res]):
                    d = np.linalg.norm(s - knots[i, :], axis=1) / theta
                    phi[:, i + K] = np.where(
                        (d >= 0) & (d <= 1),
                        (1 - d) ** 6 * (35 * d**2 + 18 * d + 3) / 3,
                        0
                    )
                K += num_basis[res]
            return phi

        phi_train = compute_basis(s_train, num_basis, knots_1d)
        phi_test = compute_basis(s_test, num_basis, knots_1d)

        y_train = df_train["tgh"].values
        y_test = df_test["tgh"].values
        
        start_time = time.time()
        model = model_function(df_train, phi_train, y_train, sim)
        train_time = time.time() - start_time

        y_pred = model.predict(phi_test).flatten()

        mae = np.mean(np.abs(y_test - y_pred))
        mse = np.mean((y_test - y_pred) ** 2)
        
        print(f"[Sim {sim+1}] g={g_val}, h={h_val} | MAE: {mae:.4f}, MSE: {mse:.4f}, Time: {train_time:.2f} sec")
        
        mae_list.append(mae)
        mse_list.append(mse)
        time_list.append(train_time)

    # Final output
    avg_time = np.mean(time_list)
    print(f"Average training time: {avg_time:.2f} sec")

    df = pd.DataFrame({
        "MAE": mae_list,
        "MSE": mse_list,
        "Time": time_list
    })

    # Add average as final row
    df.loc["Average"] = [np.mean(mae_list), np.mean(mse_list), avg_time]
    df.to_csv("results.csv", index=True)


In [31]:
Deepkriging_mae(0,0)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

##### End of warning messages ######
<<<<<<<<<<<<<<<< Fitting DNN-model for    1-th simulation >>>>>>>>>>>>>>>>>
12/12 [==============================] - 0s 582us/step
[Sim 1] g=0, h=0 | MAE: 0.1430, MSE: 0.0331, Time: 24.15 sec


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

...

0batch [00:00, ?batch/s]

##### End of warning messages ######
<<<<<<<<<<<<<<<< Fitting DNN-model for   99-th simulation >>>>>>>>>>>>>>>>>
12/12 [==============================] - 0s 630us/step
[Sim 99] g=0, h=0 | MAE: 0.1382, MSE: 0.0310, Time: 26.86 sec


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

##### End of warning messages ######
<<<<<<<<<<<<<<<< Fitting DNN-model for  100-th simulation >>>>>>>>>>>>>>>>>
12/12 [==============================] - 0s 712us/step
[Sim 100] g=0, h=0 | MAE: 0.1265, MSE: 0.0263, Time: 23.37 sec
Average training time: 26.08 sec


In [32]:
df2 = pd.read_csv("results.csv")
np.mean(df2['MAE'])
#np.std(df2['MAE'],ddof=1)/10
np.mean(df2['Time'])

26.077490923404692